# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 读取数据

导入数据分析需要的库，并通过Pandas库的`read_csv`函数，将原始数据文件`"titles.csv"`里的数据内容，解析为DataFrame并将其赋值给变量`original_titles`。将原始数据文件`"credits.csv"`里的数据内容，解析为DataFrame并将其赋值给变量`original_credits`。

In [1]:
import pandas as pd

In [2]:
original_titles=pd.read_csv("titles.csv")
original_credits=pd.read_csv("credits.csv")

In [3]:
original_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [4]:
original_credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


## 评估和清理数据

接下来将对上一部分建立的`original_titles`和`original_credits`两个DataFrame所包含的数据进行评估和清理。  

主要从两个方面进行：结构和内容，也就是整齐度和干净度。  
数据的结构性问题是指不符合“1.每一列是一个变量；2.每一行是一个观察值；3.每个单元格是一个值”这三个标准；数据的内容性问题包括丢失数据、重复数据、无效数据等。  

为了区分开清理过的数据和原始数据，我们创建新的变量`cleaned_titles`，让它成为`original_titles`的副本，以及创建新的变量`cleaned_credits`，让它成为`original_credits`的副本。之后的清理步骤都将被运用到`cleaned_titles`和`cleaned_credits`上。

In [5]:
cleaned_titles=original_titles.copy()
cleaned_credits=original_credits.copy()

### 数据整齐度

In [6]:
cleaned_titles.sample(n=10,random_state=24)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
4867,tm983723,The Sparks Brothers,MOVIE,Take a musical odyssey through five weird and ...,2021,R,140,"['documentation', 'music', 'comedy', 'history']","['GB', 'US']",NaN,tt8610436,7.8,4663.0,6.784,7.3
725,tm53625,Inventing David Geffen,MOVIE,"Notoriously press and camera-shy, David Geffen...",2012,NaN,120,['documentation'],[],NaN,NaN,NaN,NaN,2.531,7.8
5070,tm912100,Bruised,MOVIE,Jackie Justice is a mixed martial arts fighter...,2021,R,129,"['drama', 'sport', 'thriller']","['GB', 'US']",NaN,tt8310474,6.1,14293.0,29.914,7.0
1758,tm288010,Final Score,MOVIE,When a stadium is seized by a group of heavily...,2018,R,104,"['action', 'drama', 'thriller']","['GB', 'US']",NaN,tt5254610,5.7,11967.0,17.411,6.1
5609,tm1000296,New Gods: Nezha Reborn,MOVIE,"Three thousand years ago, the world was in tur...",2021,NaN,116,"['animation', 'action', 'fantasy']",['CN'],NaN,tt13269670,6.8,3150.0,3.078,8.2
3015,ts80445,The Honeymoon Stand Up Special,SHOW,Impending parenthood does funny things to Nata...,2018,NaN,32,['comedy'],[],1.0,tt7698408,6.8,519.0,1.251,6.0
2886,tm413082,Recovery Boys,MOVIE,"In the heart of America's opioid epidemic, fou...",2018,NaN,90,['documentation'],['US'],NaN,tt8167872,6.7,720.0,5.645,6.6
5053,ts222864,Falling Into Your Smile,SHOW,Student Tong Yao makes two vows: to never be i...,2021,TV-14,44,"['drama', 'comedy', 'romance']",['CN'],1.0,tt11290960,8.4,2267.0,39.102,9.0
3999,tm847623,Rogue Warfare: Death of a Nation,MOVIE,After rescuing Daniel from the terrorist Black...,2020,R,103,"['war', 'action']",['US'],NaN,tt7983744,3.3,380.0,3.373,4.8
2495,tm307751,Jab Harry Met Sejal,MOVIE,"Haunted by the memories of home he once knew, ...",2017,PG,200,"['drama', 'romance', 'comedy']",['IN'],NaN,tt10453170,5.0,20593.0,8.529,6.5


从随机抽取的10行数据来看，`genres`和`production_coutries`两个变量中包含多个值，应当进行拆分。  

先提取任意一个`genres`变量进行观察。

In [7]:
cleaned_titles["genres"][1]

"['drama', 'crime']"

虽然`genres`的表示形式是列表，但实际上它并不是字符串列表，而是字符串类型，无法直接用`value_counts`来统计各个值出现的次数。  

可以使用python内置的`eval`函数，它可以把字符串转换成表达式，所以可以把列表形式的字符串转换为列表本身。

In [8]:
cleaned_titles["genres"]=cleaned_titles["genres"].apply(lambda s:eval(s))
cleaned_titles["genres"][1]

['drama', 'crime']

转换为列表后，可以使用DataFrame的`explode`方法，把该列的列表值拆分为单独的行。

In [9]:
cleaned_titles=cleaned_titles.explode("genres")
cleaned_titles.sample(n=10,random_state=45)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
2599,tm366973,Tom Segura: Disgraceful,MOVIE,Tom Segura gives voice to the sordid thoughts ...,2018,PG-13,74,comedy,['US'],NaN,tt7379330,7.5,3851.0,4.896,7.000
3373,ts89282,Toy Boy,SHOW,"After seven years in a Málaga prison, a male s...",2019,TV-MA,61,crime,['ES'],2.0,tt8693266,6.5,7526.0,23.996,7.700
536,tm237621,The Figurine (Araromire),MOVIE,"Two buddies and a girl, down on their luck, ha...",2009,PG-13,122,thriller,['NG'],NaN,tt1542960,4.4,251.0,1.525,4.200
3319,tm316311,Wasp Network,MOVIE,"Havana, Cuba, 1990. René González, an airplane...",2020,R,128,crime,"['BR', 'FR', 'ES', 'BE']",NaN,tt6834140,5.9,10368.0,10.195,6.100
1754,tm244174,Okja,MOVIE,A young girl named Mija risks everything to pr...,2017,PG-13,122,drama,"['KR', 'US']",NaN,tt3967856,7.3,116305.0,25.911,7.415
5404,ts272515,Dealer,SHOW,Tensions erupt when two filmmakers infiltrate ...,2021,TV-MA,41,crime,['FR'],1.0,tt13702252,6.1,671.0,3.650,7.000
1357,ts55559,Ask the Storybots,SHOW,"Based on the award-winning educational apps, t...",2016,TV-Y,26,family,['US'],3.0,tt5846856,8.4,878.0,7.471,8.400
1078,ts32752,Two Fathers,SHOW,"Seven years ago, two men were notified that th...",2013,TV-14,44,comedy,['TW'],1.0,tt6707142,8.0,59.0,1.762,7.000
1981,ts80519,Spy Kids: Mission Critical,SHOW,"In this animated spinoff series, Juni and Carm...",2018,TV-Y7,25,comedy,['US'],3.0,tt6714408,4.7,382.0,7.221,8.400
3412,tm465724,See You Yesterday,MOVIE,As two teen prodigies try to master the art of...,2019,NaN,86,crime,['US'],NaN,tt8743064,5.2,10725.0,7.583,5.600


接下来，针对`production_countries`也是一样的操作。  

先提取任意一个`production_countries`变量进行观察。

In [10]:
cleaned_titles["production_countries"][1758]

1758    ['GB', 'US']
1758    ['GB', 'US']
1758    ['GB', 'US']
Name: production_countries, dtype: object

可以看出`production_countries`变量也有同样的问题，它并不是一个列表，而是列表形式的字符串，我们可以使用`eval`函数将字符串转换为表达式，也就是列表。并观察转换后的数据类型。

In [11]:
cleaned_titles["production_countries"]=cleaned_titles["production_countries"].apply(lambda s:eval(s))
cleaned_titles["production_countries"][1758]

1758    [GB, US]
1758    [GB, US]
1758    [GB, US]
Name: production_countries, dtype: object

将变量`prodection_countries`转换为列表后，就可以调用DataFrame的`explode`方法，将该列的列表值拆分为单独的行。

In [12]:
cleaned_titles=cleaned_titles.explode("production_countries")
cleaned_titles.sample(n=10,random_state=45)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
219,ts21465,Supernatural,SHOW,"When they were boys, Sam and Dean Winchester l...",2005,TV-14,45,thriller,US,15.0,tt0460681,8.4,434081.0,388.093,8.278
1205,tm135106,In a Valley of Violence,MOVIE,The story of a drifter named Paul who arrives ...,2016,R,104,western,US,NaN,tt3608930,6.0,18987.0,13.597,6.100
1578,tm149212,Olmo and the Seagull,MOVIE,'Olmo and the Seagull' is a poetic and existen...,2015,NaN,90,documentation,SE,NaN,tt4160256,7.2,676.0,2.034,7.100
1140,tm205890,The Take,MOVIE,Michael Mason is an American pickpocket living...,2016,R,92,drama,FR,NaN,tt2368619,6.3,43449.0,17.563,6.300
4954,ts302363,Twenty Five Twenty One,SHOW,The dreams of two young people are crushed by ...,2022,TV-14,76,romance,KR,1.0,tt17513352,8.7,5746.0,57.970,8.600
3291,tm824477,Over the Moon,MOVIE,"In this animated musical, a girl builds a rock...",2020,PG,95,fantasy,CN,NaN,tt7488208,6.3,22034.0,75.251,7.300
5784,ts285471,The Wedding Coach,SHOW,Former bride and forever comedian Jamie Lee sh...,2021,TV-MA,33,reality,US,1.0,tt14210424,5.0,87.0,1.525,10.000
4229,ts225377,Taj Mahal 1989,SHOW,"In and around Lucknow University in 1989, coup...",2020,NaN,34,drama,IO,1.0,tt11725706,7.4,1542.0,2.624,7.300
2272,tm311995,The Battleship Island,MOVIE,"During the Japanese colonial era, roughly 400 ...",2017,PG-13,132,drama,KR,NaN,tt5969696,7.1,7095.0,27.413,7.200
5248,tm817075,Furioza,MOVIE,An event from the past separates the fate of t...,2021,NaN,139,crime,PL,NaN,tt10515864,6.2,4129.0,14.047,6.200


在处理完`cleaned_titles`的结构性问题后，查看`cleaned_credits`。

In [13]:
cleaned_credits.sample(n=10,random_state=56)

,person_id,id,name,character,role
17554,95459,tm181081,Keith Fleming,Removal Man,ACTOR
4013,303469,tm189171,Antonio Birabent,Mauricio 'Moris' Birabent,ACTOR
30533,583772,tm315340,Salvita Decorte,Shinta,ACTOR
55471,138710,tm467804,Yogi Babu,Maari,ACTOR
8321,9573,tm81270,Har Mar Superstar,Dancin' Rick,ACTOR
3287,1100860,tm191772,Tenzin Gyaltsen Rinpoche,Tibetan,ACTOR
12056,74175,tm57586,Shriya Saran,Anjali Sagar,ACTOR
53858,1164032,ts104483,Meenacshi Martins,Maria Lopes,ACTOR
24490,98310,ts35463,Lauren Taylor,Harper Rich,ACTOR
34430,698851,ts57985,Freddy Carter,Pin,ACTOR


从随机抽取的10行数据来看，`cleaned_credits`数据符合“每列为一个变量，每行为一个观测值，每个单元格是一个值”的标准，因此不存在结构性问题。

### 数据干净度

通过`info`，来对数据内容进行大致了解。

In [14]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64

从输出结果来看，`cleaned_titles`数据共有17818条观察值，其中`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量都存在缺失数据，在后续步骤中会进行评估和处理。

另外`release_year`变量表示年份，数据类型应该为日期时间而不是数字，所以需要进行数据类型转换。

In [15]:
cleaned_titles["release_year"]=pd.to_datetime(cleaned_titles["release_year"],format="%Y")
cleaned_titles["release_year"]

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

接下来通过`info`来对`cleaned_credits`数据的内容进行大致了解。

In [16]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


从输出结果来看，`cleaned_credits`数据共有77801条观测值，其中`character`变量存在缺失值，将在后续步骤中进行评估和处理。

另外，`person_id`变量表示演职员ID，应该是字符串类型而不是数字，所以要进行数据类型转换。

In [17]:
cleaned_credits["person_id"]=cleaned_credits["person_id"].astype("str")
cleaned_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

#### 处理缺失数据

已知在`cleaned_titles`中，`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量都存在缺失数据。   

根据各变量的含义和我们的分析目标，影视作品的标题、描述、适龄认证、发行国家、电视剧季数、IMDB的ID、IMDB的投票数、TMDB的流行度和TMDB的评分都不影响我们对不同流派高分作品演员的分析。所以可以保留`title`、`description`、`age_certification`、`production_countries`、`seasons`、`imdb_id`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量存在空缺值的观察值。  

但流派和IMBD评分，也就是`genres`和`imdb_score`变量与我们后续的分析息息相关。  

首先取出`imdb_score`变量的缺失观察值进行查看。


In [18]:
cleaned_titles[cleaned_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021-01-01,TV-14,24,comedy,NG,1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


由于缺失分析所需的核心数据`imdb_scores`，我们对这些观察值进行删除，并检查删除后缺失值的个数。

In [19]:
cleaned_titles=cleaned_titles.dropna(subset=["imdb_score"])

In [20]:
cleaned_titles["imdb_score"].isnull().sum()

0

下面提取出`genres`变量有缺失数据的观察值进行查看。

In [21]:
cleaned_titles[cleaned_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018-01-01,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018-01-01,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018-01-01,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018-01-01,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020-01-01,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019-01-01,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


由于缺失分析所需的核心数据`genres`，对这些存在缺失数据的观察值进行删除，并检查删除后缺失值的个数。

In [22]:
cleaned_titles=cleaned_titles.dropna(subset=["genres"])
cleaned_titles["genres"].isnull().sum()

0

下面对`cleaned_credits`的缺失值进行评估。

`cleaned_credits`中只有表示角色姓名的`character_name`变量存在缺失值，这一变量并不影响挖掘出各个流派中的高评分作品演员，并且角色姓名缺失可能是由于`role`变量演职员类型中的值是导演而不存在角色姓名，所以可以保留`character_name`变量存在缺失值的观测值。

#### 处理重复数据

从变量的含义来看，`cleaned_titles`中不应该存在每个变量都相同的观察值，查看`cleaned_titles`中是否存在这样的观察值。

In [23]:
cleaned_titles.duplicated().sum()

0

输出结果为0，说明`cleaned_titles`中不存在每个变量都相同的观察值。  

接下来查看`cleaned_credits`中是否存在重复值。

In [24]:
cleaned_credits.duplicated().sum()

0

输出结果为0，说明`cleaned_credits`不存在每个变量都相同的观察值。

#### 处理不一致数据

在`cleaned_titles`中，存在多个不同值指代一个变量的情况可能出现在`genres`和`production_countires`中。

首先查看`genres`变量中是否存在不一致数据。

In [25]:
cleaned_titles["genres"].value_counts()

drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: genres, dtype: int64

从输出结果来看，并没有多个值指代同一流派的情况，所以不存在不一致数据。  

接下来对`production_countries`也进行同样的操作。先使用`value_counts`对`production_countries`中不同值的数量进行统计。

In [26]:
cleaned_titles["production_countries"].value_counts()

US    5648
IN    1610
GB    1068
JP    1046
FR     720
      ... 
GT       1
CU       1
LK       1
NP       1
FO       1
Name: production_countries, Length: 108, dtype: int64

由于`value_counts`执行结果中有太多值，Pandas只会默认显示开头和结尾的一些值。要完整展示结果，可以把`display.max_rows`设置为`None`，即取消展示行数上限。

但因为我们只是在当前调用`value_counts`时才需要看完整结果，所以可以结合`option_context`，只更改临时上限。  

按照代表国家名称的值来排序更方便观察是否存在不一致值，使用`sort_index`对结果进行排序。

In [27]:
with pd.option_context("display.max_rows",None):
    print(cleaned_titles["production_countries"].value_counts().sort_index())

AE           52
AF            1
AL            6
AO            3
AR          150
AT           28
AU          217
BD            3
BE          148
BF            2
BG           31
BR          221
BS            3
BT            2
BY            2
CA          608
CD            7
CH           33
CL           35
CM            3
CN          295
CO           94
CU            1
CY            5
CZ           22
DE          383
DK           89
DZ            3
EG           93
ES          637
FI           18
FO            1
FR          720
GB         1068
GE            1
GH            6
GL            3
GR            4
GT            1
HK          102
HR            3
HU           18
ID          149
IE           49
IL           42
IN         1610
IO            4
IQ            5
IR            7
IS           28
IT          224
JO           19
JP         1046
KE            6
KG            2
KH           10
KN            4
KR          637
KW           10
LB           70
LK            1
LT            7
LU      

从输出结果来看，出品国家都是由两位的国家代码来表示的，除了存在的一个`Lebanon`值。  

`Lebanon`的国家代码是`LB`，`Lebanon`出现了1次，`LB`出现了70次，说明出现了不一致数据。  

需要将`cleaned_titles`中，`production_countries`的`Lebanon`和`LB`统一为`LB`，并检查替换后是否还存在`Lebanon`。

In [28]:
cleaned_titles["production_countries"]=cleaned_titles["production_countries"].replace({"Lebanon":"LB"})

In [29]:
with pd.option_context("display.max_rows",None):
    print(cleaned_titles["production_countries"].value_counts().sort_index())

AE      52
AF       1
AL       6
AO       3
AR     150
AT      28
AU     217
BD       3
BE     148
BF       2
BG      31
BR     221
BS       3
BT       2
BY       2
CA     608
CD       7
CH      33
CL      35
CM       3
CN     295
CO      94
CU       1
CY       5
CZ      22
DE     383
DK      89
DZ       3
EG      93
ES     637
FI      18
FO       1
FR     720
GB    1068
GE       1
GH       6
GL       3
GR       4
GT       1
HK     102
HR       3
HU      18
ID     149
IE      49
IL      42
IN    1610
IO       4
IQ       5
IR       7
IS      28
IT     224
JO      19
JP    1046
KE       6
KG       2
KH      10
KN       4
KR     637
KW      10
LB      71
LK       1
LT       7
LU      27
MA      15
MC       4
MT       8
MU       5
MW       2
MX     264
MY      30
NG     131
NL     102
NO      68
NP       1
NZ      27
PE      26
PH     192
PK       9
PL     126
PR       9
PS      32
PT      14
PY       3
QA      24
RO      25
RS       2
RU      41
SA      28
SE      81
SG      47
SN       6

在`cleaned_credits`中，`role`可能存在不一致值，仍然使用`value_counts`来查看是否存在不一致值。

In [30]:
cleaned_credits["role"].value_counts().sort_index()

ACTOR       73251
DIRECTOR     4550
Name: role, dtype: int64

从输出结果来看，`role`中并不存在不一致值。  

`role`的值只有`ACTOR`和`DIRECTOR`两种，可以将其转换为`category`类型，这样比字符串类型更节省空间，也可以说明值的类型有限。

In [31]:
cleaned_credits["role"]=cleaned_credits["role"].astype("category")
cleaned_credits["role"]

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

#### 处理错误或者无效数据

可以使用DataFrame的`describe`方法，对数值统计信息进行快速了解。

In [32]:
cleaned_titles.describe()

,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
std,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831
min,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000


从输出结果来看，并不存在脱离现实意义的数据。  

`cleaned_credits`中不存在表示数值意义的变量，因此无需使用`describe`进行检查。

## 整理数据

In [33]:
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021-01-01,NaN,134,drama,NaN,NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


In [34]:
cleaned_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


对数据的整理，与分析方向紧密相关。此次数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。  

为了能同时获得流派与演员数据，我们需要把`cleaned_titles`和`cleaned_credits`，通过`id`作为键进行连接，因为两个数据表中`id`都是影视作品ID。

In [35]:
credits_with_titles=pd.merge(cleaned_credits,cleaned_titles,on="id",how="inner")

连接后，我们就能知道演职员参与过的影视作品的信息。

In [36]:
credits_with_titles.head()

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179


由于我们只对挖掘演员的参演作品感兴趣，导演不在我们分析的范围内，所以根据`role`筛选出类型为`ACTOR`的观察值，以备后续分析。

In [37]:
actor_with_titles=credits_with_titles.query("role=='ACTOR'")

为了挖掘各个流派中高IMBD评分的演员，我们需要按照流派和演员进行分组。  

由于演员姓名可能会有重复或者拼写错误，所以按照是唯一标识的演职员ID`person_id`进行分组，这样能够更加准确。

In [38]:
groupby_genres_and_person_id=actor_with_titles.groupby(["genres","person_id"])

分组后，我们只需要对`imdb_score`的值进行聚合运算，提取`imdb_score`变量，调用`mean`，计算各个流派的影视作品中，每位演员参演作品的平均IMBD评分。

In [39]:
imdb_score_groupby_genres_and_person_id=groupby_genres_and_person_id["imdb_score"].mean()
imdb_score_groupby_genres_and_person_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

我们可以调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame。

In [40]:
imdb_score_groupby_genres_and_person_id=imdb_score_groupby_genres_and_person_id.reset_index()
imdb_score_groupby_genres_and_person_id

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


现在针对流派和演员分组的IMDB评分数据已经整理好，可以进入后续的分析步骤了。  

但我们当前可以继续做一些数据整理，比如说对上面的结果进行再次分组，找到各个流派中演员作品的最高平均分是多少、最高平均分对应的演员名字是什么。  
要得到这个结果我们需要再次按照`genres`进行分组，然后提取出`imdb_score`变量，计算最大值。

In [41]:
genres_max_scores=imdb_score_groupby_genres_and_person_id.groupby("genres")["imdb_score"].max()
genres_max_scores

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

将得到的最高平均分数据，与之前的`imdb_score_groupby_genres_and_person_id`进行连接，得到最高平均分对应的`person_id`，我们就可以知道最高平均分作品对应的演员是谁。

In [42]:
genres_max_scores_with_person_id=pd.merge(imdb_score_groupby_genres_and_person_id,genres_max_scores,on=["genres","imdb_score"])
genres_max_scores_with_person_id

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


从以上结果可以看出，某个流派获得最高平均评分的演员不一定只有一位，可能有多位演员的平均得分相同。  

为了得到演员ID所对应的演员名字，我们可以和`cleaned_credits`这个DataFrame进行连接。这个DataFrame还有其它列，我们只需要得到`person_id`和`name`的对应，所以可以先提取出那两列，并把重复行删除。

In [43]:
actor_id_with_names=cleaned_credits[["person_id","name"]].drop_duplicates()
actor_id_with_names.head(10)

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
5,32267,Peter Boyle
6,519612,Leonard Harris
7,29068,Diahnne Abbott
8,519613,Gino Ardito
9,3308,Martin Scorsese


接下来就可以把`genres_max_socres_with_person_id`与`actor_id_with_names`进行连接，增加`name`变量，得到最高平均分对应演员的名字。

In [44]:
genres_max_socres_with_actor_name=pd.merge(genres_max_scores_with_person_id,actor_id_with_names,on="person_id")
genres_max_socres_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,scifi,12790,9.3,Olivia Hack
2,action,1303,9.3,Jessie Flower
3,animation,1303,9.3,Jessie Flower
4,family,1303,9.3,Jessie Flower
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka


为了将相同的流派排到一起，我们可以使用`sort_values`方法，把结果里的`genres`进行排序，然后使用`reset_index`给索引重新排序。

In [45]:
genres_max_socres_with_actor_name=genres_max_socres_with_actor_name.sort_values("genres").reset_index()
genres_max_socres_with_actor_name

,index,genres,person_id,imdb_score,name
0,0,action,12790,9.3,Olivia Hack
1,12,action,336830,9.3,André Sogliuzzo
2,7,action,21033,9.3,Zach Tyler
3,17,action,86591,9.3,Cricket Leigh
4,2,action,1303,9.3,Jessie Flower
...,...,...,...,...,...
131,131,war,826547,8.8,Yuto Uemura
132,133,western,28166,8.9,Megumi Hayashibara
133,134,western,28180,8.9,Unsho Ishizuka
134,132,western,22311,8.9,Koichi Yamadera


这时多出一列`index`列，我们把它删除。

In [46]:
genres_max_socres_with_actor_name.drop("index",axis=1)

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,336830,9.3,André Sogliuzzo
2,action,21033,9.3,Zach Tyler
3,action,86591,9.3,Cricket Leigh
4,action,1303,9.3,Jessie Flower
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,28166,8.9,Megumi Hayashibara
133,western,28180,8.9,Unsho Ishizuka
134,western,22311,8.9,Koichi Yamadera


现在我们就得到了各个流派获得最高平均评分的演员数据，为下一步分析做好准备。